In [1]:
import pandas as pd
import numpy as np
from scipy import io
import os, errno
import glob

In [2]:
#parameters to clean up data

visit_length = 150 # anything less than 150 frames gets dropped
# m = 0.4 # mean value of Prob(proboscis)

frames_btw_visits = 600

In [3]:
def MergeVisits(df, frames):
    idx = df[df['frames_inbetween'] <= frames].index.values
    if len(idx)>1:
        conseq = idx[:-1] - idx[1:]
        for c, ind in zip(conseq, idx[:-1]):
            if c is -1:
                new_val = df.loc[idx+1,'Out_Frame'].values
                df.loc[idx- 1, 'Out_Frame'] = new_val 
            else:
                new_val = df.loc[idx,'Out_Frame'].values
                df.loc[idx- 1, 'Out_Frame'] = new_val 
        if c != -1:
            new_val = df.loc[idx[-1],'Out_Frame']
            df.loc[idx[-1]- 1, 'Out_Frame'] = new_val
    else:
        new_val = df.loc[idx,'Out_Frame'].values
        df.loc[idx- 1, 'Out_Frame'] = new_val

    df = df.drop(idx, axis = 0)
    df = df.reset_index(drop = True)
    return(df)

In [4]:
direc1 = r"G:\My Drive\Mahad\Light Level Experiments\Data Output from Analysis\Step 2/"
filelist = glob.glob(direc1 + '*MothInOut.csv')

In [6]:
# get the csv files with manual entry of num Trials entered by experimentor 
direc2 = r"G:\My Drive\Mahad\Light Level Experiments\Back up of experiment folder/"
notes = pd.read_excel(direc2 + 'MothChart-LightLevel-FlowerShape.xlsx',
                     sheet_name = 0)
notes.head()

,Date,Animal Name,Light Level,Eclosed Date,Date Caught,"Age (at time of expt, in days)",Gender,Body Length (cm),Proboscis Length (cm),Total trials,Successful trials,Comments,Person,Unnamed: 13
0,2018_07_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,L3_c-2_m1,2.85,2018_7_28-29,2018_07_31,2 or 3,Male,4.7,7.8,21.0,19.0,"Cover for light fell off during experiment, re...",Mahad,NaN
2,NaN,L0.3_c-3_m1,0.30,2018_7_28-29,2018_07_31,2 or 3,Female,4.8,9.0,26.0,26.0,NaN,Mahad,NaN
3,NaN,L0.03_c-1_m1,0.04,2018_7_28-29,2018_07_31,2 or 3,Male,4.7,8.0,0.0,0.0,NaN,Mahad,NaN
4,2018_08_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
notes.columns

Index(['Date', 'Animal Name', 'Light Level', 'Eclosed Date', 'Date Caught',
       'Age (at time of expt, in days)', 'Gender', 'Body Length (cm)',
       'Proboscis Length (cm)', 'Total trials', 'Successful trials',
       'Comments', 'Person', 'Unnamed: 13'],
      dtype='object')

In [8]:
new_path = r"./dataFolders/Output/Step3_v2/"

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [9]:
num_trial = pd.DataFrame(data = None, columns = ['name', 'initial_num', 'dlc_filter', 'total_len_filter', 'final_num', 'notesExpts'], dtype = int)
num_trial.head()

,name,initial_num,dlc_filter,total_len_filter,final_num,notesExpts


In [10]:
for i, file in enumerate(filelist):
    df = pd.read_csv(file)
    total_trial = len(df)
    
    a,b, c, _ = os.path.basename(file).split("_")
    name = a + "_" + b + "_" + c
        
#    # Get the stats on DLCTracking
#    file_matched = [f for f in file_dlc if os.path.basename(file)[:-14] in f]
#    temp = pd.read_hdf(file_matched[0])
    
#    DLC = temp.loc[slice(None),(slice(None),'proboscisTip')].copy()
#    DLC.columns = DLC.columns.droplevel(level = [0,1])

#    mean_col = []
    
    for row in df.iterrows():
        data = row[1]

        In = data.In_Frame
        out = data.Out_Frame

#        mu = DLC.loc[In:out, 'likelihood'].mean(skipna = True)
#        mean_col.append(mu)
#    df.loc[:,'mean'] = mean_col

    #drop trials based  on length and prob(proboscis)
#    filtered_df = df[df['mean'] > m].copy()
    filtered_df = df.copy()
    mean_filt_trials = np.nan #len(filtered_df)
    
    filt_df = filtered_df[filtered_df.Frames_In_View > visit_length]
    filt_df = filt_df.reset_index(drop = True)
    dwell_time_trials = len(filt_df)
    
    #merge trials
    frames_inbetween =  filt_df.In_Frame.values[1:] - filt_df.Out_Frame.values[:-1]
    frames_inbetween = frames_inbetween.astype(np.float64)
    frames_inbetween = np.insert(frames_inbetween, 0, np.nan, axis = 0)
    filt_df['frames_inbetween'] = frames_inbetween
    
    final_df = MergeVisits(filt_df, frames_btw_visits)
    final_trials = len(final_df)
    
     # find the total visits as noted by the experimentor
    tt = notes[notes['Animal Name'] == name]['Total trials'].values
    
    # save the TRUE moth In and Out
#     final_df = final_df.drop(columns = ['mean', 'frames_inbetween', 'FramesInView'])
    final_df = final_df.drop(columns = ['frames_inbetween', 'Frames_In_View'])
    final_df.to_csv(new_path + name + '_trueTrials.csv')
      
    num_trial.loc[i, 'name'] = name
    num_trial.loc[i,'initial_num'] = total_trial
    num_trial.loc[i,'dlc_filter'] = mean_filt_trials
    num_trial.loc[i,'total_len_filter'] = dwell_time_trials
    num_trial.loc[i,'final_num'] = final_trials
    num_trial.loc[i, 'notesExpts'] = tt
    
num_trial.to_csv(new_path + 'trialSelectionStats.csv')